In [1]:
import pandas as pd
import numpy as np
import re
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

In [2]:
# Step 1: Load the Dataset
file_path = "/Users/mohammadadnaan/Downloads/Prodigal_Tech/intelligent-debt-recovery-genai/src/ml_models/privacy_compliance_detection/conversations_with_labels.xlsx"
df = pd.read_excel(file_path)

# Display the first few rows of the dataset
print(df.head())

                                Call ID  \
0  2db2965e-54fa-41fa-823b-ed79b943f0b1   
1  8a9655a7-be88-4921-b0ad-04aa1b9953d1   
2  19169ec6-213f-48e9-8ec2-c24ee2e6eb20   
3  c86714f0-3aeb-4628-8cfa-ee0a46839508   
4  0c8297aa-ced1-414e-a175-bf29a9763d30   

                                                Text  Label  \
0  Hello, this is Emma from XYZ Collections, how ...      0   
1  Hello, this is Mark from XYZ Collections. How ...      0   
2  Hello, this is Mark calling from XYZ Collectio...      0   
3  Hello, this is Mark calling from XYZ Collectio...      0   
4  Hello, this is Mike from XYZ Collections. Am I...      0   

   Double Verification  
0                    0  
1                    1  
2                    0  
3                    1  
4                    1  


In [3]:
# Step 2: Preprocess the Text Data
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Remove special characters and numbers
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    
    # Lemmatization (using simple rule-based approach)
    lemmatized_text = []
    for word in text.split():
        if word.endswith('ing'):
            word = word[:-3]  # Remove 'ing'
        elif word.endswith('ed'):
            word = word[:-2]  # Remove 'ed'
        elif word.endswith('s'):
            word = word[:-1]  # Remove 's'
        lemmatized_text.append(word)
    text = " ".join(lemmatized_text)
    
    # Remove stop words (using a custom list)
    stop_words = set(["the", "and", "is", "in", "it", "to", "of", "for", "with", "on", "at", "by", "this", "that", "are", "as", "be", "was", "were", "you", "your", "we", "our"])
    text = " ".join([word for word in text.split() if word not in stop_words])
    
    return text

# Apply preprocessing to the text column
df["Processed_Text"] = df["Text"].apply(preprocess_text)


In [4]:
# Step 3: Tokenization and Padding
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(df["Processed_Text"])

with open('/Users/mohammadadnaan/Downloads/Prodigal_Tech/intelligent-debt-recovery-genai/src/ml_models/privacy_compliance_detection/tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)  # Save FIRST

# Convert text to sequences
sequences = tokenizer.texts_to_sequences(df["Processed_Text"])
max_len = 100
padded_sequences = pad_sequences(sequences, maxlen=max_len, padding="post")

In [5]:
# Step 4: Prepare Labels
labels = df["Label"].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

In [6]:
# Step 5: Build the LSTM Model
embedding_dim = 100  # Dimension of word embeddings
vocab_size = len(tokenizer.word_index) + 1  # Vocabulary size

model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_shape=(max_len,)),  # Add input_shape
    LSTM(128, dropout=0.2, recurrent_dropout=0.2),
    Dense(64, activation="relu"),
    Dropout(0.5),
    Dense(1, activation="sigmoid")  # Binary classification
])

# Compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Display the model summary
model.summary()

/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 100, 100)       │        94,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 128)            │       117,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 219,569 (857.69 KB)

 Trainable params: 219,569 (857.69 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
# Step 6: Train the Model
history = model.fit(X_train, y_train, epochs=200, batch_size=32, validation_split=0.2)

Epoch 1/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 80ms/step - accuracy: 0.7047 - loss: 0.6769 - val_accuracy: 0.9500 - val_loss: 0.5896
Epoch 2/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.8753 - loss: 0.5318 - val_accuracy: 0.9500 - val_loss: 0.1840
Epoch 3/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.8745 - loss: 0.3978 - val_accuracy: 0.9500 - val_loss: 0.2707
Epoch 4/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.8506 - loss: 0.3636 - val_accuracy: 0.9500 - val_loss: 0.3044
Epoch 5/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.8432 - loss: 0.3824 - val_accuracy: 0.9500 - val_loss: 0.2259
Epoch 6/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.8441 - loss: 0.3278 - val_accuracy: 0.9500 - val_loss: 0.1610
Epoch 7/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.9104 - loss: 0.2388 - val_accuracy: 0.9500 - val_loss: 0.1618
Epoch 8/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.8588 - loss: 0.2737 - val_accuracy: 0.9500 - val_loss:

In [8]:
# Step 7: Evaluate the Model
# Predict on the test set
y_pred = (model.predict(X_test) > 0.5).astype("int32")

# Print classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Print accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.96      0.93        45
           1       0.33      0.20      0.25         5

    accuracy                           0.88        50
   macro avg       0.62      0.58      0.59        50
weighted avg       0.86      0.88      0.87        50

Accuracy: 88.00%


In [9]:
# Step 8: Save the Model
model.save("privacy_compliance_lstm_model.keras")
print("Model saved successfully.")

Model saved successfully.


In [10]:
import numpy as np

# Example message to test
message = ['Your account has been flagged for suspicious activity. Please call us immediately to resolve the issue.']

# Step 1: Tokenize and pad the message
seq = tokenizer.texts_to_sequences(message)  # Convert text to sequence of integers
padded = pad_sequences(seq, maxlen=max_len, dtype='int32', value=0)  # Pad the sequence

# Step 2: Make a prediction with the model
pred = model.predict(padded)  # Get the prediction probability

# Define labels
labels = ['No issue (Label 0)', 'Sensitive info shared without verification (Label 1)']

# Get the predicted class (0 or 1)
predicted_class = int(pred[0][0] > 0.5)  # Threshold at 0.5 for binary classification

# Step 4: Show the predicted class with its probability
print(f"Prediction: {labels[predicted_class]} with probability: {pred[0][0] * 100:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Prediction: No issue (Label 0) with probability: 0.00%


In [17]:
message = ['Hello, this is Mark from XYZ Collections. How are you today? What the hell do you want? I have told you assholes to leave me alone! I understand that you may be upset. I am calling about your account with ABC Bank. Can I have your account number to assist you? Why the fuck would I give you anything? You are just some low-life calling me! I assure you, I am here to help resolve this matter. Your current balance is $500. How do you prefer to settle this? Settle this? You guys are a bunch of crooks! I do not owe you shit! I understand your frustration. However, it is important for us to discuss this. The account is past due. Fuck you! You think I am gonna pay you just because you say so? I appreciate your honesty. My goal is simply to find a solution. Would you be open to setting up a payment plan? A payment plan? You are dreaming! Not a dime from me! I respect your decision. Just know I am here to assist whenever you are ready to discuss options. Yeah, whatever. Just get the hell off my phone! Thank you for your time. Have a good day.']

# Step 1: Tokenize and pad the message
seq = tokenizer.texts_to_sequences(message)  # Convert text to sequence of integers
padded = pad_sequences(seq, maxlen=max_len, dtype='int32', value=0)  # Pad the sequence

# Step 2: Make a prediction with the model
pred = model.predict(padded)  # Get the prediction probability

# Define labels
labels = ['No issue (Label 0)', 'Sensitive info shared without verification (Label 1)']

# Get the predicted class (0 or 1)
predicted_class = int(pred[0][0] > 0.5)  # Threshold at 0.5 for binary classification

# Step 4: Show the predicted class with its probability
print(f"Prediction: {labels[predicted_class]} with probability: {pred[0][0] * 100:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Prediction: Sensitive info shared without verification (Label 1) with probability: 100.00%


In [13]:
message = ['Hello this is Max from ABC credit bank. Am I talking to Adam? Yes you are. You son of bitch why did not you pay the last installment of your debt. I would like to talk for this matter professionally and respectfully. Okay Whats your Date of Birth and address for respecting your  privacy? its 14th January 1990 and I am living on Adam block, high rise apartment , new york. Thanks your due amount is 500$.']

# Step 1: Tokenize and pad the message
seq = tokenizer.texts_to_sequences(message)  # Convert text to sequence of integers
padded = pad_sequences(seq, maxlen=max_len, dtype='int32', value=0)  # Pad the sequence

# Step 2: Make a prediction with the model
pred = model.predict(padded)  # Get the prediction probability

# Define labels
labels = ['No issue (Label 0)', 'Sensitive info shared without verification (Label 1)']

# Get the predicted class (0 or 1)
predicted_class = int(pred[0][0] > 0.5)  # Threshold at 0.5 for binary classification

# Step 4: Show the predicted class with its probability
print(f"Prediction: {labels[predicted_class]} with probability: {pred[0][0] * 100:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
Prediction: No issue (Label 0) with probability: 0.00%


In [16]:
message = ['Hello, this is Mark from XYZ Collections. How are you today? I am okay, thanks. What is this about? I am calling regarding your outstanding debt with Global Bank. Can you confirm your name for me? It is Jordan Smith. Thank you, Jordan. Your account shows a balance of $750. How would you like to settle this today? I need to check my finances before deciding. I understand. Just so you know, we can set up a payment plan if that helps. That could work. Can you send me the details? Absolutely, I can send that via email. Can you confirm your email address? It is jordan.smith@email.com. Great! I will send the details right away. Is there anything else you need? No, that is all for now. Thanks. You are welcome! Have a great day. You too. Bye!']

# Step 1: Tokenize and pad the message
seq = tokenizer.texts_to_sequences(message)  # Convert text to sequence of integers
padded = pad_sequences(seq, maxlen=max_len, dtype='int32', value=0)  # Pad the sequence

# Step 2: Make a prediction with the model
pred = model.predict(padded)  # Get the prediction probability

# Define labels
labels = ['No issue (Label 0)', 'Sensitive info shared without verification (Label 1)']

# Get the predicted class (0 or 1)
predicted_class = int(pred[0][0] > 0.5)  # Threshold at 0.5 for binary classification

# Step 4: Show the predicted class with its probability
print(f"Prediction: {labels[predicted_class]} with probability: {pred[0][0] * 100:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Prediction: No issue (Label 0) with probability: 0.00%
